In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from scipy.fft import fft
from sklearn.svm import SVC

In [2]:
data_df=pd.read_csv('/Users/mm/Desktop/Myolab/prepared_data.csv')

In [3]:
data_df.columns

Index(['exercise', 'rep_count_from_intermediate', 'rep_count_from_start',
       'ref_xy_rotation', 'time', 'euler_X', 'euler_Y', 'euler_Z'],
      dtype='object')

In [4]:
data_df.shape

(1188661, 8)

In [5]:
numerical_features = data_df[['euler_X', 'euler_Y','euler_Z','rep_count_from_intermediate', 'ref_xy_rotation',]]  
categorical_features = data_df['exercise']  # Target variable

# Encode the categorical target variable
encoder = OneHotEncoder(sparse_output=False)
target_encoded = encoder.fit_transform(categorical_features.values.reshape(-1, 1))

In [6]:
scaler = StandardScaler()
numerical_features_scaled = scaler.fit_transform(numerical_features)

In [7]:
num_samples = numerical_features_scaled.shape[0]
num_time_steps = 1 
num_features = numerical_features_scaled.shape[1]

X_reshaped = numerical_features_scaled.reshape((num_samples, num_time_steps, num_features))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, target_encoded, test_size=0.2, random_state=42)

In [9]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(num_time_steps, num_features)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(target_encoded.shape[1], activation='softmax') 
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/30
29717/29717 [==============================] - 39s 1ms/step - loss: 1.1612 - accuracy: 0.5329 - val_loss: 0.9397 - val_accuracy: 0.6102
Epoch 2/30
29717/29717 [==============================] - 36s 1ms/step - loss: 1.0132 - accuracy: 0.5805 - val_loss: 0.8911 - val_accuracy: 0.6293
Epoch 3/30
29717/29717 [==============================] - 37s 1ms/step - loss: 0.9794 - accuracy: 0.5942 - val_loss: 0.8652 - val_accuracy: 0.6413
Epoch 4/30
29717/29717 [==============================] - 36s 1ms/step - loss: 0.9597 - accuracy: 0.6030 - val_loss: 0.8462 - val_accuracy: 0.6500
Epoch 5/30
29717/29717 [==============================] - 36s 1ms/step - loss: 0.9452 - accuracy: 0.6091 - val_loss: 0.8312 - val_accuracy: 0.6582
Epoch 6/30
29717/29717 [==============================] - 36s 1ms/step - loss: 0.9357 - accuracy: 0.6135 - val_loss: 0.8187 - val_accuracy: 0.6662
Epoch 7/30
29717/29717 [==============================] - 36s 1ms/step - loss: 0.9266 - accuracy: 0.6174 - val_loss: 0

In [11]:
performance = model.evaluate(X_test, y_test)

7430/7430 [==============================] - 3s 373us/step - loss: 0.6516 - accuracy: 0.7475


In [12]:
predictions = model.predict(X_test)

7430/7430 [==============================] - 3s 352us/step
